In [1]:
# 引入函示庫
import numpy as np
import pandas as pd
import math
import glob

In [31]:
real_distance = 1.824
for file in glob.glob('../Data/20200430_mix/Processed/csv/imu_0430_4/IMU_1.csv'):
    data = pd.read_csv(file)
    data['x_v'] = np.abs(data['x'].iloc.cumsum()*data['duration'])/1000
    data['y_v'] = np.abs(data['y'].cumsum()*data['duration'])/1000
    data['z_v'] = np.abs(data['z'].cumsum()*data['duration'])/1000
    data['x_distance'] = data['x_v'].cumsum()*data['duration']
    data['y_distance'] = data['y_v'].cumsum()*data['duration']
    data['z_distance'] = data['z_v'].cumsum()*data['duration']
    data['z_error'] = np.abs(real_distance-data['z_distance'])
    data['z_error']

In [34]:
file = '../Data/20200430_mix/Processed/csv/imu_0430_4/IMU_1.csv'
data = pd.read_csv(file)
data['x_v'] = data['x'].cumsum()*data['duration']/1000
data['y_v'] = data['y'].cumsum()*data['duration']/1000
data['z_v'] = data['z'].cumsum()*data['duration']/1000
data['x_distance'] = data['x_v'].cumsum()*data['duration']
data['y_distance'] = data['y_v'].cumsum()*data['duration']
data['z_distance'] = data['z_v'].cumsum()*data['duration']
data['z_error'] = np.abs(real_distance-data['z_distance'])

In [35]:
data['z_distance'].iloc[-1]

-391509542.64906794

In [5]:
# 加速度積分
x_v = np.zeros(data_num,float) # x軸速度
y_v = np.zeros(data_num,float) # y軸速度
z_v = np.zeros(data_num,float) # z軸速度
for i in range(0,data_num):
    x_v[i] = np.abs((np.sum(data[0][i:i+2]))*data[3][i]/sample_rate) # x軸加速度對時間積分 => x軸速度
    y_v[i] = np.abs((np.sum(data[1][i:i+2]))*data[3][i]/sample_rate) # y軸加速度對時間積分 => y軸速度
    z_v[i] = np.abs((np.sum(data[2][i:i+2]))*data[3][i]/sample_rate) # z軸加速度對時間積分 => z軸速度

In [6]:
distance = 14.62 # 實際測量距離
time = np.sum(data[3])/1000 # 實際測量時間
real_distance = round(2*distance/time,2) #實際間隔位移
print('實際測量距離 :',distance,'m')
print('實際測量時間 :',time,'s')
print('實際間隔位移 :',real_distance,'m')
interval = np.zeros(sample_num,float) # 間隔數
x_distance = np.zeros(sample_num,float) # x軸間隔位移
y_distance = np.zeros(sample_num,float) # y軸間隔位移
z_distance = np.zeros(sample_num,float) # z軸間隔位移
error_x = np.zeros(sample_num,float) # x軸位移誤差
error_y = np.zeros(sample_num,float) # y軸位移誤差
error_z = np.zeros(sample_num,float) # z軸位移誤差
for i in range(0,sample_num):
    interval[i] = i+1
    x_distance[i] = np.sum(x_v[(i+1)*sample_rate:(i+2)*sample_rate])/20
    y_distance[i] = np.sum(y_v[(i+1)*sample_rate:(i+2)*sample_rate])
    z_distance[i] = np.sum(z_v[(i+1)*sample_rate:(i+2)*sample_rate])/20
    error_x[i] = np.abs(round((0-x_distance[i])*100,2))
    error_y[i] = np.abs(round((real_distance-y_distance[i])*100,2))
    error_z[i] = np.abs(round((0-z_distance[i])*100,2))
    print('第',int(interval[i]),'間隔位移 :',round(y_distance[i],4),'m','誤差 :',error_y[i],'cm')

實際測量距離 : 14.62 m
實際測量時間 : 13.027 s
實際間隔位移 : 2.24 m
第 1 間隔位移 : 1.9469 m 誤差 : 29.31 cm
第 2 間隔位移 : 1.7839 m 誤差 : 45.61 cm
第 3 間隔位移 : 1.8967 m 誤差 : 34.33 cm
第 4 間隔位移 : 1.8605 m 誤差 : 37.95 cm


In [7]:
# 輸出excel檔案
result = list(zip(interval,x_distance,error_x,y_distance,error_y,z_distance,error_z))
result_df = pd.DataFrame(result,columns = ['間隔數','x間隔位移(m)','x誤差(cm)','y間隔位移(m)','y誤差(cm)','z間隔位移(m)','z誤差(cm)'])
with pd.ExcelWriter('y_distance_1.xlsx') as writer:
    result_df.to_excel(writer, sheet_name = '加速規位移', index = False)